# Parameters

This page focuses on the topic of how the API's parameters are displayed in the documentation. We will check it mainly as results in `openapi.json`, assuming that engines for display will show everything.

In [1]:
import json
import requests
import pandas as pd

Create the initial state of the application.

In [2]:
%%writefile parameters_files/app.py
from fastapi import FastAPI

app = FastAPI()

@app.get("/")
def example(par):
    return "hello"

Overwriting parameters_files/app.py


Run the docker container that uses the file created below.

In [3]:
%%bash
docker run --rm -itd\
    --name test_container\
    -v ./parameters_files/app.py:/app.py\
    -p 8000:8000 \
    fastapi_experiment \
    uvicorn --host 0.0.0.0 --reload app:app >/dev/null
sleep 10

## Get parameters

In order to obtain the parameters that require a handler for a specific path, you need to refer to the keys `paths/<considered path>/<http method>/parameters`. The provided code cell showcases a portion of the `openapi.json` file extracted from the basic example, which corresponds to the `/` handler.

In [6]:
json.loads(
    requests.get("http://localhost:8000/openapi.json").content.decode("utf-8")
)["paths"]["/"]

{'get': {'summary': 'Example',
  'operationId': 'example__get',
  'parameters': [{'name': 'par',
    'in': 'query',
    'required': True,
    'schema': {'title': 'Par'}}],
  'responses': {'200': {'description': 'Successful Response',
    'content': {'application/json': {'schema': {}}}},
   '422': {'description': 'Validation Error',
    'content': {'application/json': {'schema': {'$ref': '#/components/schemas/HTTPValidationError'}}}}}}}

## Required parameters

I'm interested in how the various options associated with the required parameters for the api are displayed in the docs.

The following cell creates different handlers that create parameters in different syntax:

- `/ex1` - uses syntax of path params:
    - `par1` - parameter that is not needed anywhere;
    - `par2` - parameter that is required only by path;
    - `par3` - parameter is required only by function;
    - `par4` - parameter is required for both.
- `/ex2` - uses syntax of query params:
    - `par1` - is required;
    - `par2` - not required.

In [7]:
%%writefile parameters_files/app.py
from fastapi import FastAPI

app = FastAPI()

@app.get("/ex1/{par1}{par2=10}{par3}{par4=10}")
def ex1(par1:int, par2:int, par3=10, par4=10):
    return "hello"

@app.get("/ex2")
def ex2(par1:int, par2=10):
    return "hello"

Overwriting parameters_files/app.py


Ok now let's check what we see in the `parameters` for different paths.

In [8]:
pd.concat(
    {
        path : pd.DataFrame(
            json.loads(
                requests.get("http://localhost:8000/openapi.json").content.decode("utf-8")
            )["paths"][path]["get"]["parameters"]
        )
        for path in [
            "/ex1/{par1}{par2=10}{par3}{par4=10}",
            "/ex2"
        ]
    },
    names=["paths"]
)

name     in  required  \
paths                                                          
/ex1/{par1}{par2=10}{par3}{par4=10} 0  par1   path      True   
                                    1  par3   path      True   
                                    2  par2  query      True   
                                    3  par4  query     False   
/ex2                                0  par1  query      True   
                                    1  par2  query     False   

                                                                     schema  
paths                                                                        
/ex1/{par1}{par2=10}{par3}{par4=10} 0  {'type': 'integer', 'title': 'Par1'}  
                                    1                     {'title': 'Par3'}  
                                    2  {'type': 'integer', 'title': 'Par2'}  
                                    3      {'default': 10, 'title': 'Par4'}  
/ex2                                0  {'type': 'integer', 'title': 'Par1'}  
                                    1      {'default': 10, 'title': 'Par2'}

The main focus here is that to create an optional parameter using path params, you need to set it as required in both path and function. It's enough to specify it in the function arguments to make it required by the api.

## Stop container

Don't forget to stop the container you created at the beginning.

In [9]:
!docker stop test_container

test_container
